# Importing Packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# Creating read_csv Function

In [2]:
def read_my_data(csv_file):
    my_data = pd.read_csv(csv_file)
    return my_data

In [3]:
my_data = read_my_data("python_assignment_1.csv")

# Tidying Data

In [ ]:
my_data.columns = my_data.columns.str.title()
my_data = my_data.replace(['positiveprime', 'negativeprime', 'positivetarget', 'negativetarget'], 
                ['Positive', 'Negative', 'Positive', 'Negative'])

In [ ]:
my_data.head()

In [ ]:
def run_my_anova(Participant, IV1, IV2, DV, data):
    mapping = {data.columns[0]: Participant, data.columns[1]: IV1, data.columns[2]: IV2, data.columns[3]: DV}
    data_frame = data.rename(columns=mapping)
    descriptives = data_frame[[IV1, IV2, DV]].groupby([IV1,IV2]).describe()
    return descriptives

In [ ]:
run_my_anova('Participant', 'Prime', 'Target', 'RT', my_data)

In [ ]:
my_data.head()

# Creating Visualisation

# Going beyond

In [ ]:
IV1 = input('Enter your first independent variable: ')
IV2 = input('Enter your second independent variable: ')
DV = input('Enter your dependent variable: ')
mapping = {my_data.columns[1]: IV1, my_data.columns[2]: IV2, my_data.columns[3]: DV}
data = my_data.rename(columns=mapping)
data.columns = data.columns.str.title()
data

In [ ]:
tidy_data = input('Would you like to tidy your data? y/n: ')
if tidy_data == 'y':
    level_1_origin = input('What would you like to rename: ')
    level_1_change = input('What would you like to rename it to: ')
    data = data.replace([level_1_origin],[level_1_change])
    carry_on = input('Would you like to rename a second variable? y/n: ')
    if carry_on == 'y':
        level_2_origin = input('What would you like to rename: ')
        level_2_change = input('What would you like to rename it to: ')
        data = data.replace([level_2_origin],[level_2_change])
        carry_on = input('Would you like to rename a third variable? y/n: ')
    if carry_on == 'y':
        level_3_origin = input('What would you like to rename: ')
        level_3_change = input('What would you like to rename it to: ')
        data = data.replace([level_3_origin],[level_3_change])
        carry_on = input('Would you like to rename a final variable? y/n: ')
    if carry_on == 'y':
        level_4_origin = input('What would you like to rename: ')
        level_4_change = input('What would you like to rename it to: ')
        data = data.replace([level_4_origin],[level_4_change])
        carry_out_summary = input('Would you like to generate some summary statistics? y/n: ')
    elif carry_on == 'n':
        carry_out_summary = input('Would you like to generate some summary statistics? y/n: ')
elif tidy_data == 'n':
    carry_out_summary = input('Would you like to generate some summary statistics? y/n: ')

In [7]:
IV1 = input('Enter your first independent variable: ')
IV2 = input('Enter your second independent variable: ')
DV = input('Enter your dependent variable: ')
mapping = {my_data.columns[1]: IV1, my_data.columns[2]: IV2, my_data.columns[3]: DV}
data = my_data.rename(columns=mapping)
data.columns = data.columns.str.title()
data.head()
tidy_data = input('Would you like to tidy your data? y/n: ')
if tidy_data == 'y':
    level_1_origin = input('What would you like to rename: ')
    level_1_change = input('What would you like to rename it to: ')
    data = data.replace([level_1_origin],[level_1_change])
    carry_on = input('Would you like to rename a second variable? y/n: ')
    if carry_on == 'y':
        level_2_origin = input('What would you like to rename: ')
        level_2_change = input('What would you like to rename it to: ')
        data = data.replace([level_2_origin],[level_2_change])
        carry_on = input('Would you like to rename a third variable? y/n: ')
    if carry_on == 'y':
        level_3_origin = input('What would you like to rename: ')
        level_3_change = input('What would you like to rename it to: ')
        data = data.replace([level_3_origin],[level_3_change])
        carry_on = input('Would you like to rename a final variable? y/n: ')
    if carry_on == 'y':
        level_4_origin = input('What would you like to rename: ')
        level_4_change = input('What would you like to rename it to: ')
        data = data.replace([level_4_origin],[level_4_change])
        carry_out_summary = input('Would you like to generate some summary statistics? y/n: ')
    elif carry_on == 'n':
        carry_out_summary = input('Would you like to generate some summary statistics? y/n: ')
elif tidy_data == 'n':
    carry_out_summary = input('Would you like to generate some summary statistics? y/n: ')

Enter your first independent variable: Prime
Enter your second independent variable: Target
Enter your dependent variable: rt


KeyboardInterrupt: Interrupted by user

In [ ]:
carry_out_summary = input('Would you like to generate some summary statistics? y/n: ')
if carry_out_summary == 'y':
    data_frame[[IV1, IV2, DV]].groupby([IV1,IV2]).describe()

NameError: name 'view' is not defined